# PRMT-2477 Pre GP2GP failures from MI data

## Context

We have been doing some research with practices to understand issues that occur during registrations that prevent GP2GP from happening. There are known scenarios such as patients coming from abroad, but we want to see if there are unknown scenarios that also contribute, e.g. to do with PDS or SDS issues. 

Questions to be answered using MI data:

For a given month (Oct or Nov)

1. How many registrations failed with any of the following process failure points:

- 10 = PDS trace
- 20 = PDS update
- 30 = SDS lookup Practice (not used)
- 40 = SDS lookup ASID

2. Are there any registrations that have any of these failure points and eventually go to GP2GP (i.e. have a conversation ID?)

3. Do these process failure points correlate with any of the specific failure types? : 

- 0 = Attempted,
- 1 = Sent,
- 2 = Not Sent - Patient at current practice,
- 3 = Not Sent - Patient known at current practice transferring from non-GP2GP practice,
- 4 = Not Sent - Patient not known at current practice transferring from a non-GP2GP practice,
- 5 = Not Sent – Patient has no previous practice registered,
- 6 = Negative acknowledgement received.

4. Can we tell which registrations are failed but could have gone via GP2GP, vs. which are not eligible for GP2GP e.g. new born, coming from Scotland or Wales, Army, prison, International etc. 

## Notes

Data downloaded from Splunk using the following query:
```
index="gp2gp_nms_prod" sourcetype="gp2gpmi-rr"
| table *
```

In [1]:
import pandas as pd
import numpy as np
import paths, data
from data.practice_metadata import read_asid_metadata

In [2]:
def convert_to_float(val):
    try:
        return int(val)
    except:
        return val

mi_data_file_location = "s3://prm-gp2gp-notebook-data-prod/PRMT-2477-pre-gp2gp-failures/MI_RR-Nov_2021.csv"

dates_fields = ["RegistrationTime", "RequestFailureTime", "RequestTime", "ExtractTime", "ExtractAckTime", "ExtractAckFailureTime"]
practice_registrations = pd.read_csv(mi_data_file_location, parse_dates=dates_fields)

practice_registrations["RequestErrorCode"] = practice_registrations["RequestErrorCode"].apply(convert_to_float)
practice_registrations = practice_registrations.fillna("None")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
practice_registrations["UniqueKey"] = (practice_registrations["RegistrationSmartcardUID"]
                                       + "-"
                                       + practice_registrations["RegistrationTime"].astype(str))

practice_registrations = (
    practice_registrations
        .sort_values(by="_time", ascending=True)
        .drop_duplicates(subset=["UniqueKey"], keep="last")
    )

In [4]:
# Check how many registrations have no Smartcard ID
practice_registrations_without_smartcard_bool = practice_registrations["RegistrationSmartcardUID"]=="None"
practice_registrations_without_smartcard = practice_registrations[practice_registrations_without_smartcard_bool]
practice_registrations_without_smartcard.shape[0]

1

In [5]:
# Check total number of registrations
practice_registrations.shape[0]

553282

In [6]:
# Check how many registrations have no conversation ID
practice_registrations["ConversationID"].value_counts()

None                                    316708
1799d21b-d5fb-447f-9952-036f3b640bca         2
5de434d4-b050-4487-a249-2681fe856daa         2
7c97eb78-5031-4490-92e8-24b28546cd2e         2
d984f259-90d3-4354-ab42-1890da0d0e5c         2
                                         ...  
32aa09f4-f2b1-4e8e-9a94-202e20058666         1
a3b52eeb-59fd-4112-8a5c-c2489c67c05f         1
F005EC40-4211-11EC-B073-5F862D02F9B7         1
7a46079e-fa87-44af-8c0b-647688289088         1
eef15528-1976-47df-89d8-2617d87288b2         1
Name: ConversationID, Length: 236558, dtype: int64

In [7]:
# Breakdown of all registrations that did not trigger GP2GP
def has_conversation_id(value):
    if value=="None":
        return 0
    else:
        return 1
    
practice_registrations["TriggeredGP2GP"] = practice_registrations.apply(lambda row: has_conversation_id(row["ConversationID"]), axis=1)

did_not_trigger_gp2gp_bool = practice_registrations["TriggeredGP2GP"]==False
practice_registrations_no_gp2gp = practice_registrations[did_not_trigger_gp2gp_bool]

registrations_that_didnt_trigger_gp2gp_grouped_by_failures = (
    practice_registrations_no_gp2gp
        .groupby(by=["RequestFailurePoint", "RequestFailureType", "RequestErrorCode", "TriggeredGP2GP"])
        .agg({"RegistrationTime": "count"})
        .rename(columns={"RegistrationTime": "count"})
        .sort_values(by="count", ascending=False)
    )
registrations_that_didnt_trigger_gp2gp_grouped_by_failures

count
RequestFailurePoint RequestFailureType RequestErrorCode TriggeredGP2GP        
60                  5.0                None             0               149446
                    0.0                None             0                79387
0                   0.0                None             0                49610
10                  None               20               0                22395
60                  2.0                None             0                 7466
40                  3.0                24               0                 3142
                    4.0                24               0                 2749
20                  0.0                20               0                 1121
60                  0.0                20               0                  793
40                  None               20               0                  509
20                  0.0                IU030            0                   70
                                       IU056            0                   10
                                       None             0                    5
                                       -3               0                    2
                                       IU052            0                    2
                                       IU066            0                    1

In [8]:
# Breakdown of all registrations with specific request failure points (in list below)

failure_points_of_interest = [10, 20, 30, 40]
is_failure_point_of_interest = practice_registrations["RequestFailurePoint"].apply(lambda error_code: error_code in failure_points_of_interest)
registrations_with_failure_points_of_interest = practice_registrations[is_failure_point_of_interest]

all_registrations_grouped_by_failures = (
    registrations_with_failure_points_of_interest
        .groupby(by=["RequestFailurePoint", "RequestFailureType", "RequestErrorCode", "TriggeredGP2GP"])
        .agg({"RegistrationTime": "count"})
        .rename(columns={"RegistrationTime": "count"})
        .sort_values(by="count", ascending=False)
    )
all_registrations_grouped_by_failures

count
RequestFailurePoint RequestFailureType RequestErrorCode TriggeredGP2GP       
10                  None               20               0               22395
40                  3.0                24               0                3142
                    4.0                24               0                2749
20                  0.0                20               0                1121
40                  None               20               0                 509
20                  0.0                IU030            0                  70
                                       -8               1                  38
                                       IU056            0                  10
                                       None             0                   5
                                       -3               0                   2
                                       25               1                   2
                                       IU052            0                   2
                                       IU066            0                   1

In [9]:
# Breakdown of specific request failure points per practice
def has_error_scenario(row, scenarioList):
    if row["RequestFailurePoint"]==scenarioList[0] and row["RequestFailureType"]==scenarioList[1] and row["RequestErrorCode"]==scenarioList[2]:
        return 1
    else:
        return 0
    
# error scenario format below is like so: [RequestFailurePoint, RequestFailureType, RequestErrorCode]
errorScenariosOfInterest = [
    [10, "None", 20],
    [40, 3, 24],
    [40, 4, 24],
    [20, 0, 20],
    [40, "None", 20]
]

practice_registrations["ErrorScenario_10_None_20"] = practice_registrations.apply(lambda row: has_error_scenario(row, errorScenariosOfInterest[0]), axis=1)
practice_registrations["ErrorScenario_40_3_24"] = practice_registrations.apply(lambda row: has_error_scenario(row, errorScenariosOfInterest[1]), axis=1)
practice_registrations["ErrorScenario_40_4_24"] = practice_registrations.apply(lambda row: has_error_scenario(row, errorScenariosOfInterest[2]), axis=1)
practice_registrations["ErrorScenario_20_0_20"] = practice_registrations.apply(lambda row: has_error_scenario(row, errorScenariosOfInterest[3]), axis=1)
practice_registrations["ErrorScenario_40_None_20"] = practice_registrations.apply(lambda row: has_error_scenario(row, errorScenariosOfInterest[4]), axis=1)

practice_registrations_with_pivot = (practice_registrations.groupby('RequestorODS')
                                     .agg({"UniqueKey": "count",
                                           "TriggeredGP2GP": "sum",
                                           "ErrorScenario_10_None_20": "sum",
                                           "ErrorScenario_40_3_24": "sum",
                                           "ErrorScenario_40_4_24": "sum",
                                           "ErrorScenario_20_0_20": "sum",
                                           "ErrorScenario_40_None_20": "sum"})
                                     .rename(columns={"UniqueKey": "Total Registrations"})
                                     .fillna(0)
                                     .astype(int)
                                     .sort_values(by="ErrorScenario_10_None_20", ascending=False))
practice_registrations_with_pivot.head(5)

,Total Registrations,TriggeredGP2GP,ErrorScenario_10_None_20,ErrorScenario_40_3_24,ErrorScenario_40_4_24,ErrorScenario_20_0_20,ErrorScenario_40_None_20
RequestorODS,,,,,,,
F81141,930,23,845,1,0,11,0
E87768,2049,283,443,9,0,1,0
E85124,2869,1264,372,10,0,7,0
M85124,313,18,250,3,0,0,0
H82012,355,41,215,0,0,0,0


In [10]:
# Add practice names (via ASID lookup) to the table above
asid_lookup = read_asid_metadata("prm-gp2gp-ods-metadata-preprod", "v2/2021/12/organisationMetadata.json")[["practice_ods_code", "practice_name"]]
asid_lookup = asid_lookup.set_index("practice_ods_code")

practice_registrations_with_pivot_with_practice_names = practice_registrations_with_pivot.join(asid_lookup, on="RequestorODS", how="left")
practice_registrations_with_pivot_with_practice_names = practice_registrations_with_pivot_with_practice_names.drop_duplicates()
practice_registrations_with_pivot_with_practice_names = practice_registrations_with_pivot_with_practice_names.reset_index().rename(columns={"practice_name": "Practice Name", "RequestorODS": "Requesting Practice ODS"})

columns_order = ["Requesting Practice ODS", "Practice Name", "Total Registrations", "TriggeredGP2GP", "ErrorScenario_10_None_20", "ErrorScenario_40_3_24", "ErrorScenario_40_4_24", "ErrorScenario_20_0_20", "ErrorScenario_40_None_20"]
practice_registrations_with_pivot_with_practice_names = practice_registrations_with_pivot_with_practice_names[columns_order]

In [11]:
with pd.ExcelWriter("PRMT-2477-Practice-level-pre-GP2GP-error-scenarios-Nov-2021.xlsx") as writer:
     practice_registrations_with_pivot_with_practice_names.to_excel(writer, sheet_name="Pre GP2GP errors Nov 2021",index=False)

**RequestFailurePoint:**
- 0 = No failure
- 10 = PDS trace
- 20 = PDS update

- 30 = SDS lookup Practice (not used)
- 40 = SDS lookup ASID
- 50 = SDS lookup Contract Props
- 60 = Send Request
- 70 = Manual Request

**RequestFailureType:**
- 0 = Attempted
- 1 = Sent
- 2 = Not Sent - Patient at current practice
- 3 = Not Sent - Patient known at current practice transferring from non-GP2GP practice
- 4 = Not Sent - Patient not known at current practice transferring from a non-GP2GP practice
- 5 = Not Sent – Patient has no previous practice registered
- 6 = Negative acknowledgement received

**RequestErrorCode:**
- 3 = Record available but cannot be sent - DEPRECATED
- 8 = The system’s configuration prevents it from processing this message - DEPRECATED
- 20 = Spine system responded with an error
- 24 = SDS lookup provided zero or more than one result to the query for each interaction
- 25 = Large messages rejected due to timeout duration reached of overall transfer